In [50]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D
from keras.datasets import imdb
from keras import backend as K

In [63]:
max_features = 5000
maxlen = 100
batch_size = 32
embedding_dims = 50
out_dim = 250
window = 3
hidden_dims = 250
nb_epoch = 2

In [52]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features,
                                                      test_split=0.2)

In [53]:
X_train[0]

[1,
 20,
 28,
 716,
 48,
 495,
 79,
 27,
 493,
 8,
 2,
 7,
 50,
 5,
 4682,
 2,
 10,
 5,
 852,
 157,
 11,
 5,
 1716,
 3351,
 10,
 5,
 500,
 2,
 6,
 33,
 256,
 41,
 2,
 7,
 17,
 23,
 48,
 1537,
 3504,
 26,
 269,
 929,
 18,
 2,
 7,
 2,
 4284,
 8,
 105,
 5,
 2,
 182,
 314,
 38,
 98,
 103,
 7,
 36,
 2184,
 246,
 360,
 7,
 19,
 396,
 17,
 26,
 269,
 929,
 18,
 1769,
 493,
 6,
 116,
 7,
 105,
 5,
 575,
 182,
 27,
 5,
 1002,
 1085,
 130,
 62,
 17,
 24,
 89,
 17,
 13,
 381,
 1421,
 8,
 2,
 7,
 5,
 2723,
 38,
 325,
 7,
 17,
 23,
 93,
 9,
 156,
 252,
 19,
 235,
 20,
 28,
 5,
 104,
 76,
 7,
 17,
 169,
 35,
 2,
 17,
 23,
 1460,
 7,
 36,
 2184,
 934,
 56,
 2134,
 6,
 17,
 891,
 214,
 11,
 5,
 1552,
 6,
 92,
 6,
 33,
 256,
 82,
 7]

In [54]:
len(X_train[0])

140

In [55]:
max(len(ex) for ex in X_train)

2821

In [56]:
min(len(ex) for ex in X_train)

12

In [57]:
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [58]:
len(X_train[0])

100

In [59]:
max(len(ex) for ex in X_train)

100

In [60]:
min(len(ex) for ex in X_train)

100

In [64]:
model = Sequential()

In [65]:
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))

In [66]:
model.add(Convolution1D(nb_filter=out_dim,
                        filter_length=window,
                        border_mode='valid',
                        activation='tanh',
                        subsample_length=1))

In [67]:
def max_1d(X):
    return K.max(X, axis=1)

model.add(Lambda(max_1d, output_shape=(conv_dim,)))

In [68]:
model.add(Dense(hidden_dims))

In [69]:
model.add(Activation('tanh'))

In [70]:
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [46]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [71]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
embedding_2 (Embedding)            (None, 100, 50)     250000      embedding_input_2[0][0]          
____________________________________________________________________________________________________
convolution1d_3 (Convolution1D)    (None, 98, 250)     37750       embedding_2[0][0]                
____________________________________________________________________________________________________
lambda_2 (Lambda)                  (None, 250)         0           convolution1d_3[0][0]            
____________________________________________________________________________________________________
dense_3 (Dense)                    (None, 250)         62750       lambda_2[0][0]                   
___________________________________________________________________________________________

In [49]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          validation_data=(X_test, y_test))

Train on 20000 samples, validate on 5000 samples
Epoch 1/2
20000/20000 [==============================] - 27s - loss: 0.2731 - acc: 0.8858 - val_loss: 0.3613 - val_acc: 0.8336
Epoch 2/2
20000/20000 [==============================] - 30s - loss: 0.1455 - acc: 0.9456 - val_loss: 0.4867 - val_acc: 0.8178
